In [7]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Activation, Dropout, Flatten, GlobalAveragePooling2D
from keras import optimizers
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3
import tensorflow as tf

using_gpu = tf.test.is_gpu_available()



In [8]:
IMG_SIZE = 150
LEARNING_RATE = 0.001
EPOCHS = 5
BATCH_SIZE = 32

TRAIN_SIZE = 8005
TEST_SIZE = 2023

TRAIN_DIR = '/Users/luoluo/BLS/DataBase/jpg/train_set'
TEST_DIR = '/Users/luoluo/BLS/DataBase/jpg/test_set'

In [9]:
datagen = ImageDataGenerator(rescale=1./255)

print("Training Set")
train_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=['non', 'real'],
    batch_size=BATCH_SIZE)
print("")

print("Test Set")
test_gen = datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=['non', 'real'],
    batch_size=BATCH_SIZE)

Training Set
Found 7828 images belonging to 2 classes.

Test Set
Found 2276 images belonging to 2 classes.


In [10]:
# Initialize Inception Model
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

for layer in inception_model.layers:
    layer.trainable = False

# Add top layers
x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax', name='predictions')(x)

# Combine base model with top layers
model = Model(inception_model.input, predictions)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

print("Modified Inception V3 model ready")

87924736/87910968 [==============================] - 18s 0us/step
Modified Inception V3 model ready


In [11]:
training = model.fit_generator(
    generator=train_gen,
    steps_per_epoch= TRAIN_SIZE // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_gen,
    validation_steps= TEST_SIZE // BATCH_SIZE)

/var/folders/zt/pxqlgx6n02b943rwp60qx4080000gn/T/ipykernel_10340/638570618.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  training = model.fit_generator(


Epoch 1/5
250/250 [==============================] - 884s 4s/step - loss: 0.4633 - accuracy: 0.7860 - val_loss: 0.4230 - val_accuracy: 0.8065


In [ ]:
import matplotlib.pyplot as plt  

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(16,6))

# loss history
ax0.plot(training.history['loss'])
ax0.plot(training.history['val_loss'])
ax0.set_title('Model Loss')
ax0.set_ylabel('Loss')
ax0.set_xlabel('Epoch')
ax0.legend(['Train', 'Test'], loc='upper right')

#accuracy history
ax1.plot(training.history['accuracy'])
ax1.plot(training.history['val_accuracy'])
ax1.set_title('Model Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['Train', 'Test'], loc='upper left')

plt.show()